In [84]:
import betfairlightweight as bflw
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize as jn
import os
import re
import glob
from dotenv import load_dotenv, find_dotenv
import bz2

In [37]:
# Set view parameters
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Set figsize for charting
# plt.rcParams['figure.figsize'] = (16, 8)

# Get login creds
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

username = os.environ.get('BF_USERNAME')
password = os.environ.get('BF_PASSWORD')
app_delay_key = os.environ.get('API_DELAY_KEY')
certs = '/Users/henry/code/freelance/upwork/04.03_betfair_download/certs'
dl_path = '/Users/henry/code/freelance/upwork/04.03_betfair_download/data'

In [8]:
trading = bflw.APIClient(username, password, app_key=app_delay_key, certs=certs)

trading.login()

<LoginResource>

In [9]:
# for year in [2015, 2016]:
# for month in range(1,13):
file_list = trading.historic.get_file_list(
    'Soccer', 'Basic Plan', 1, 1, 2017, 31, 1, 2017, 
    countries_collection=['GB', 'DE'], market_types_collection=['MATCH_ODDS'], file_type_collection=['M'])

# target_dir = str(year) + '_' + str(month)
target_dir = 'data/' + '2017_01'
os.makedirs(target_dir)

for file in file_list:
    download = trading.historic.download_file(file_path=file, store_directory=os.getcwd() + '/' + target_dir)

In [26]:
url = 'http://historicdata.betfair.com/api/DownloadListOfFiles'
headers = {'content-type': 'application/json','ssoid': app_delay_key} 
payload = {
    "sport":"Soccer",
    "plan":"Basic Plan",
    "fromDay" : 1,
    "fromMonth": 1,
    "fromYear" : 2016,
    "toDay": 31,
    "toMonth" : 1,
    "toYear": 2016,
    "marketTypesCollection": ["MATCH_ODDS" ],
    "countriesCollection" : ["GB", "DE" ],
    "fileTypeCollection" : ["M"]
}

In [27]:
r = requests.get(url, headers=headers, params=payload)

------------------------------------------------------------------------------------
------------------------------------------------------------------------------------

### Test repeats

In [6]:
path = '/Users/henry/code/freelance/upwork/04.03_betfair_download/data'

In [95]:
# General - set all .bz2 files in data folder as 'files'
file_paths = glob.glob(os.path.join(path + '/**/*.bz2'), recursive=True)

In [96]:
# Remove corrupted files
for file in file_paths:
    try:
        bz2.BZ2File(file).read()
    except OSError:
        os.remove(file)
    except EOFError:
        os.remove(file)

In [97]:
# Get full file list
dlfile_list = trading.historic.get_file_list(
    'Soccer', 'Basic Plan', 1, 4, 2015, 31, 11, 2018, 
    countries_collection=['GB', 'DE'], market_types_collection=['MATCH_ODDS'], file_type_collection=['M'])

In [73]:
# Create get_stub function
def get_stub(file_path):
    try:
        stub = re.search(r'1.\d*.bz2', file_path)[0]
    except TypeError:
        stub = np.nan
    return stub

In [98]:
# Transform into df
dlfile_list_df = pd.DataFrame(dlfile_list, columns=['dlfile_name'])

# Create stub column for match
dlfile_list_df['stub'] = dlfile_list_df['dlfile_name'].apply(get_stub)

# file paths df
file_paths_df = pd.DataFrame(file_paths, columns=['file_path'])

# Create stub column for match
file_paths_df['stub'] = file_paths_df['file_path'].apply(get_stub)

# Merge dfs on stub
redl = dlfile_list_df.merge(file_paths_df, on='stub')

# Create target_dir column
redl['target_dir'] = redl['file_path'].apply(lambda x: re.search(r'201\d{1}_\d{2}', x)[0])

In [102]:
redl['target_dir'].value_counts()

2016_09    1753
2018_03    1516
2017_02    1301
2017_10    1218
2016_10    1134
2017_01    1088
2018_01    1063
2015_08    1019
2017_04    1001
2017_08     929
2017_03     881
2015_09     878
2018_02     869
2017_11     866
2018_10     843
2016_11     838
2018_09     788
2018_04     670
2018_11     593
2015_07     512
2016_08     428
2017_05     413
2015_12     410
2016_04     394
2015_10     367
2015_11     320
2016_12     292
2017_07     281
2016_01     273
2018_07     259
2017_09     257
2018_05     206
2015_05     121
2018_08     112
2016_05     109
2016_07     100
2017_06      58
2017_12      51
2016_06      48
2015_06      25
2015_04       5
Name: target_dir, dtype: int64

In [114]:
len(redl)

24289

In [101]:
failed = []

In [115]:
upto = 24289

trading = bflw.APIClient(username, password, app_key=app_delay_key, certs=certs)
trading.login()

for index, row in redl[upto-2000:upto].iterrows():

    target_dir = row['target_dir']
    try:
        download = trading.historic.download_file(file_path=row['dlfile_name'], store_directory=os.getcwd() + '/data/' + target_dir)
        print('success')
    except FileNotFoundError:
        print('fail')
        failed.append(file)

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


In [15]:
repeats = [x for x in files if x not in files]

In [17]:
files[0]

'/Users/henry/code/freelance/upwork/04.03_betfair_download/data/2017_09/1.134377774.bz2'

In [18]:
file_list[0]

'/data/xds/historic/BASIC/28000000/1.128020066.bz2'

In [16]:
len(repeats)

53197

In [26]:
len(repeats2)/len(files)

0.5261202893855955

In [25]:
len(repeats2)

24362

In [28]:
import re

In [29]:
month_regex = re.compile(r'201\d{1}_\d{2}')

In [33]:
re.search(month_regex, files[1])[0]

'2017_09'

In [45]:
months = [re.search(month_regex, x)[0] for x in repeats]

In [47]:
x = pd.Series(months)

In [48]:
x.value_counts()

2017_01    2197
2016_04    1927
2015_10    1920
2015_11    1742
2016_08    1683
2015_12    1612
2016_01    1587
2016_12    1501
2015_09    1349
2017_04    1283
2016_11     895
2016_10     836
2017_08     835
2016_07     654
2016_05     648
2017_11     639
2017_10     603
2015_08     528
2015_05     506
2017_05     443
2017_02     372
2017_07     275
2015_07     116
2017_12      80
2015_06      43
2016_06      41
2017_06      15
2017_09      14
2018_09       6
2018_03       5
2018_10       3
2018_02       2
2018_08       2
dtype: int64

In [42]:
failed[:10]

['/Users/henry/code/freelance/upwork/04.03_betfair_download/data/2017_09/1.133967104.bz2',
 '/Users/henry/code/freelance/upwork/04.03_betfair_download/data/2017_09/1.134270030.bz2',
 '/Users/henry/code/freelance/upwork/04.03_betfair_download/data/2017_09/1.133902349.bz2',
 '/Users/henry/code/freelance/upwork/04.03_betfair_download/data/2017_09/1.134240743.bz2',
 '/Users/henry/code/freelance/upwork/04.03_betfair_download/data/2017_09/1.134127012.bz2',
 '/Users/henry/code/freelance/upwork/04.03_betfair_download/data/2017_09/1.134240804.bz2',
 '/Users/henry/code/freelance/upwork/04.03_betfair_download/data/2017_09/1.134231552.bz2',
 '/Users/henry/code/freelance/upwork/04.03_betfair_download/data/2017_09/1.133561992.bz2',
 '/Users/henry/code/freelance/upwork/04.03_betfair_download/data/2017_09/1.133967026.bz2',
 '/Users/henry/code/freelance/upwork/04.03_betfair_download/data/2017_09/1.134290795.bz2']

In [40]:
len(failed)

4887

In [36]:
failed = []

for file in repeats:
    target_dir = re.search(month_regex, file)[0]
    try:
        download = trading.historic.download_file(file_path=file, store_directory=os.getcwd() + '/' + target_dir)
    except FileNotFoundError:
        failed.append(file)

ProxyError: HTTPSConnectionPool(host='historicdata.betfair.com', port=443): Max retries exceeded with url: / (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))

------------------------------------------------------------------------------------
------------------------------------------------------------------------------------

In [12]:
# Log in
trading = bflw.APIClient(username, password, app_key=app_delay_key, certs=certs)
trading.login()

<LoginResource>

In [5]:
file_list = trading.historic.get_file_list(
    'Soccer', 'Basic Plan', 1, 1, 2017, 31, 1, 2017,
    countries_collection=['GB', 'DE'], market_types_collection=['MATCH_ODDS'], file_type_collection=['M'])

# for file in file_list:
#     download = trading.historic.download_file(file_path=file)

In [3]:
uncompressedData = bz2.BZ2File('1.138807922.bz2').read()

In [43]:
uncompressedData[0]

NameError: name 'uncompressedData' is not defined

In [7]:
trading.historic.get_my_data()

StatusCodeError: Status code error: 500

In [13]:
collection_options = trading.historic.get_collection_options('Soccer', 'Basic Plan', 1, 12, 2017, 31, 12, 2017)

In [10]:
jn(collection_options)

,countriesCollection,fileTypeCollection,marketTypesCollection
0,"[{'name': '', 'count': 1794}, {'name': 'AE', '...","[{'name': 'E', 'count': 3275}, {'name': 'M', '...","[{'name': '', 'count': 3441}, {'name': 'ACCA',..."


In [13]:
jn(collection_options['countriesCollection'])

,count,name
0,1794,
1,570,AE
2,107,AG
3,75,AM
4,2926,AR
5,351,AT
6,627,AU
7,328,AZ
8,184,BA
9,575,BD


In [14]:
# Get advance basket data size
basket_size = trading.historic.get_data_size('Soccer', 'Basic Plan', 1, 8, 2017, 31, 8, 2017)
print(basket_size)

{'totalSizeMB': 277, 'fileCount': 178028}


In [15]:
file_list = trading.historic.get_file_list(
    'Soccer', 'Basic Plan', 1, 12, 2015, 31, 12, 2015,
    countries_collection=['GB', 'DE'], market_types_collection=['MATCH_ODDS'], file_type_collection=['M'])

In [14]:
for file in file_list[:5]:
    print(file)
#     download = trading.historic.download_file(file_path=file)
#     print(download)

/data/xds/historic/BASIC/28489702/1.137265248.bz2
/data/xds/historic/BASIC/28489735/1.137266407.bz2
/data/xds/historic/BASIC/28489786/1.137270391.bz2
/data/xds/historic/BASIC/28489786/1.137270318.bz2
/data/xds/historic/BASIC/28499470/1.137562893.bz2
